In [1]:
#https://umi-tools.readthedocs.io/en/latest/API.html
#pip install umi_tools
import gzip
import io
import pandas as pd
import numpy as np
import sys


########################## Get tables of expected gRNA sequences
grnalist = pd.read_csv("/corgi/otherdataset/crispr_padlock/metadata/brunello.csv",sep="\t")
set_grna_brunello = set(grnalist["sgRNA Target Sequence"])  #20bp

grnalist = pd.read_csv("/corgi/otherdataset/crispr_padlock/metadata/kinome.csv",sep="\t")
set_grna_kinome = set(grnalist["sgRNA Target Sequence"])  #20bp


all_set_grna = dict({
    "Brunello":set_grna_brunello,
    "Brunello-kinome":set_grna_kinome,
    "GFPg1":set_grna_brunello  #hack
})


########################## Read metadata to figure out what files we have
samplemeta = pd.read_csv("/corgi/otherdataset/crispr_padlock/metadata/samplemeta.csv",sep="\t")
samplemeta = samplemeta[samplemeta["protocol"]=="padlock"]

rootdir = "/husky/martin/CRISPR_padlock_all_fastq/padlock/"



subsamp for mip


/home/mahogny/miniconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/tmp/ipykernel_1843673/2863434824.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:


#########
# Read FASTQ files, extract gRNA and UMI from reads.
# Discard reads that do not perfectly match brunello list
def readFastqMIP(seqR1, seqR2, seqR1b, seqR2b,  set_grna):

    ### Extract all reads from fastq
    with io.TextIOWrapper(io.BufferedReader(gzip.open(seqR1))) as buffer:
        lines=buffer.readlines()
    allreadsR1 = [lines[x+1] for x in range(0, len(lines), 4)]

    with io.TextIOWrapper(io.BufferedReader(gzip.open(seqR2))) as buffer:
        lines=buffer.readlines()
    allreadsR2 = [lines[x+1] for x in range(0, len(lines), 4)]

    ### If resequenced, add additional reads
    if seqR1b!="":
        with io.TextIOWrapper(io.BufferedReader(gzip.open(seqR1))) as buffer:
            lines=buffer.readlines()
        allreadsR1b = [lines[x+1] for x in range(0, len(lines), 4)]

        with io.TextIOWrapper(io.BufferedReader(gzip.open(seqR2))) as buffer:
            lines=buffer.readlines()
        allreadsR2b = [lines[x+1] for x in range(0, len(lines), 4)]

        allreadsR1 = allreadsR1 + allreadsR1b
        allreadsR2 = allreadsR2 + allreadsR2b



    ### Extract the sgRNA, which is the first n letters
    grna = [r[20:(20+20)] for r in allreadsR1]

    ### Extract the UMI, which is the first n letters
    umis = [r[0:13] for r in allreadsR2]

    ## Put read & UMI together. Only keep sgRNAs in library (others missequenced etc).
    ## BWA or other aligner would do this better
    correct_readpairs = [x for x in zip(grna, umis) if x[0] in set_grna]

    table_reads = pd.DataFrame(correct_readpairs,columns=['grna','umi'])

    return table_reads


#########
# This deduplicates UMIs for all gRNAs in a data frame of (grna,umi). Need not be sorted
def dedup_full_table(table_reads):

    ###### Get a data frame with grna as rownames, and list of counts per umi as one column
    todedup = table_reads.groupby(['grna'])
    deduped = todedup.apply(umicount_per_grna_and_group)


    ###### Filter out single-count UMIs, and give a summary count of deduped gRNAs
    def countumi_and_filter(onegroup):
        return len([x for x in onegroup]) ################# disabled
        #return len([x for x in onegroup if x>1])

    counttable_deduped = pd.DataFrame({
        "grna":deduped.index,
        "cnt":[countumi_and_filter(x) for x in deduped]}
    )
    return counttable_deduped




In [3]:
i = 11


cursamp = list(samplemeta["filename"])[i]
cursamp_re = list(samplemeta["reseq_sample"])[i]
print(i)
print(cursamp)
print(cursamp_re)


11
padlock_1_R1_001.fastq.gz
nan


In [4]:

if type(cursamp_re) == str:
    fR1 = rootdir+cursamp_re
    fR2 = rootdir+cursamp_re.replace("_R1","_R2")
else:
    fR1 = ""
    fR2 = ""

print("additional reads")
print(fR1)
print(fR2)


## Read all the data, extract grna,umi
table_reads = readFastqMIP(
    rootdir+cursamp,
    rootdir+cursamp.replace("_R1","_R2"),
    fR1,
    fR2,
    all_set_grna[list(samplemeta["library"])[i]]
)
print("done reading. num reads")
print(table_reads.shape[0])
#table_reads.to_csv("/home/mahogny/mystore/dataset/crispr_padlock/umicount/"+cursamp+".raw.csv", index=False)


additional reads


done reading. num reads
849287


In [5]:
print("cnt per grna")
counts = table_reads[["grna"]].value_counts()
d = pd.DataFrame({"grna":[x[0] for x in list(counts.index)], "cnt":list(counts)})
d

cnt per grna


,grna,cnt
0,GGAATCCGGAGCTCATGAGG,1966
1,GTCCTCATCCGGTCAGGCTG,1931
2,GCCTATCGGCATTCCCACTG,1817
3,TAGCAAGCATGATCTTTCGA,1816
4,TGATCGGCCAAATCACACGT,1785
...,...,...
2087,GCATTCTGGTCGGACATCAG,1
2088,GCACCGCTTGTGGACCACGC,1
2089,TGACCGACTTCGGCCACACG,1
2090,GTCCTCATCTAGCGACTCGT,1


In [7]:
## Focus on the top most abundant grnas for speed
d=d.sort_values(by="cnt", ascending=False)
top_grnas=list(d["grna"])[0]
table_reads = table_reads[table_reads["grna"]==top_grnas]


In [8]:
table_reads

,grna,umi
952,GGAATCCGGAGCTCATGAGG,ATGCCTTGCTAAA
1859,GGAATCCGGAGCTCATGAGG,CAAAGTACCATAT
2225,GGAATCCGGAGCTCATGAGG,ATGCCTTGCTAAA
2835,GGAATCCGGAGCTCATGAGG,ATCCAGTGGTTAG
3087,GGAATCCGGAGCTCATGAGG,CGAATAAAACAAC
...,...,...
846471,GGAATCCGGAGCTCATGAGG,ACCCAACCTGAAA
847784,GGAATCCGGAGCTCATGAGG,ATGCCTTGCTAAA
847793,GGAATCCGGAGCTCATGAGG,AAACAGTGGATAG
848026,GGAATCCGGAGCTCATGAGG,ACAAACCAGTAGC


In [40]:

#########
# For a group of UMIs, look up how many counts there are for each, and return a list of counts.
# We may later wish to omit 1-count UMIs (failed to group with any)
def sum_umicounts(umicount_bytes, onegroup):
    return sum([umicount_bytes[x] for x in onegroup])



#########
# This deduplicates UMIs for a single gRNA; meant to be applied to a data frame with "umi" column
def umicount_per_grna_and_group(tab):
    #print(tab)

    ### Count occurence of each UMI according to raw sequencing data
    from collections import Counter
    umicount = dict(Counter(tab["umi"]))

    ### UMI-tools expects umis to be byte strings, not regular strings
    umicount_bytes = dict(zip(
        [bytes(k, 'ascii') for k in umicount.keys()],
        umicount.values()))

    ### Gets groups of umis belonging together, but not how many of each umi
    from umi_tools import UMIClusterer
    clusterer = UMIClusterer(cluster_method="directional")
    clustered_umis = clusterer(umicount_bytes, threshold=5) #here we have a large distance, and then we filter the bad ones later!

    #print(clustered_umis)
    #They are not in size order
    
    #Ensure sgRNA with biggest sgRNA first
    clustered_umis.sort(key=lambda x: -len(x))
    
    #print([len(x) for x in clustered_umis])

    
    #Different by how many bases from main sgRNA?
    print("main umi")
    main_umi=clustered_umis[0]
    print(main_umi)
    
    print("as char")
    print(list(main_umi))
    
    
    ### Gather counts per umi group
    return [sum_umicounts(umicount_bytes, onegroup) for onegroup in clustered_umis]


In [48]:
##############################
############################## Get the empirical distribution of UMIs for one sgRNA
##############################


def diff_letters(a,b):
    a=list(a)
    b=list(b)
    return sum ( a[i] != b[i] for i in range(len(a)) )


tab = table_reads

counts_umi = tab[["umi"]].value_counts()
d_umi = pd.DataFrame({"umi":[x[0] for x in list(counts_umi.index)], "cnt":list(counts_umi)})
d_umi = d_umi.sort_values(by="cnt", ascending=False)

main_umi = d_umi["umi"][0]

d_umi["dist"]=[diff_letters(x,main_umi) for x in d_umi["umi"]]

d_umi.to_csv("/corgi/henriksson/jupyter/crispr_padlock/umi_dist.csv")

In [22]:
tab = table_reads

### Count occurence of each UMI according to raw sequencing data
from collections import Counter
umicount = dict(Counter(tab["umi"]))

### UMI-tools expects umis to be byte strings, not regular strings
umicount_bytes = dict(zip(
    [bytes(k, 'ascii') for k in umicount.keys()],
    umicount.values()))

### Gets groups of umis belonging together, but not how many of each umi
from umi_tools import UMIClusterer
clusterer = UMIClusterer(cluster_method="directional")
clustered_umis = clusterer(umicount_bytes, threshold=5) #here we have a large distance, and then we filter the bad ones later!


#Ensure UMI with most reads first
clustered_umis.sort(key=lambda x: -len(x))


topumi_group = [s.decode("utf-8") for s in clustered_umis[0]]

In [25]:


df_umigroup = pd.DataFrame({"umi":topumi_group})

counts = df_umigroup[["umi"]].value_counts()
d = pd.DataFrame({"umi":[x[0] for x in list(counts.index)], "cnt":list(counts)})

d

d=d.sort_values(by="cnt", ascending=False)
d



,umi,cnt
0,AAAAACCAGTAGC,1
106,AAATACGAGTAGC,1
99,AAGGTATCCATCA,1
100,AAAATTAAAACTA,1
101,AAACACTGGATAG,1
...,...,...
52,TGCGTAAGAGACC,1
53,TGCTTAAGAGACA,1
54,TGCTTATGAGAAC,1
55,TTCCCGCCAAACT,1


In [41]:


###### Get a data frame with grna as rownames, and list of counts per umi as one column
todedup = table_reads.groupby(['grna'])
deduped = todedup.apply(umicount_per_grna_and_group)


main umi
[b'CAAAATCAATGTG', b'CTAAATCAATGTG', b'CTATATCAATGTG', b'CAATATCAATGTG', b'CTAGATCAATGTG', b'CTCTATCAATGTG', b'CTCAATCAATGTG', b'CTAGATGAATGTG', b'CAAAAACAATGTG', b'CTCGATGAATGTG', b'CTAAAGCAATGTG', b'CTAAATGAATGTG', b'CAACATCAATGTG', b'CAAAATGATTGTG', b'CTACATCAATGTG', b'CTGTATGAATGTG', b'CAAAATGAATGTG', b'CTATATCAATGGG', b'CTAGATCAATGGG', b'CTCCATCAATGTG', b'ATCTACTAAACCT', b'CTAGATCAATGCG', b'CTAGAGCAATGTG', b'ATCTACAATACCA', b'ATCGACTAACCCA', b'CAAAAACAAAGAG', b'TACTACTATCAAA', b'ATCTACTAACCCA', b'CACGACTCATACT', b'ACGTCAAAACAAA', b'CACGATCAATGAG', b'CTAGATGAATGAG', b'AACTACATAACAA', b'AACAACTAATCAA', b'CTGGATCAATGTG', b'CTATAGCAATGTG', b'CACCATCAATGTG', b'ATGTACAAAACAA', b'CACAAAAAATGTG', b'CTATATCATTGTG', b'CACGCCTCATACT', b'AACGACAAAACAA', b'AACTACAAAGCCA', b'CAAAATCAATATG', b'CTCAAGCAATGTG', b'CACGCCTCATCCT', b'CACAATCAATGTG', b'TACTACAAAACAA', b'ATCGACAAAACAA', b'CTCTAGCAATGTG', b'CTCTAGGAATGTG', b'CAAACACAATGTG', b'CTCAAGGAATGTG', b'CACTATCAATGTG', b'ATATACAAAACAA', 

main umi
[b'AATAGAGATAGCG', b'ATTTGAGATAGCG', b'ATTAGAGATTGCG', b'ATGTGAGATTGCG', b'ATTAGCGATAGCG', b'AAGAGAGATAGCG', b'ATGTGAGATAGCG', b'ATGAGAGATAGCG', b'ATTTGAGATTGCG', b'ATGAGAGATTGCG', b'ATTTGCGATTGCG', b'ATTTGCGATAGCG', b'AATAGCGATAGCG', b'ATGTGCGATTGCG', b'ATTAGCGATTGCG', b'ATGAGCGATAGCG', b'TATACAGATAGCG', b'ATGTGCGATAGCG', b'AATAGAGCTAGCG', b'ATGAGCGATTGCG', b'TTTTCCTTAGGCA', b'ATTGTCTGCGCCT', b'ATTGACTGCGCCG', b'ATTCACTGACCCT', b'AATAGAGAAAGCG', b'ATTCACGGCCCCT', b'ATTGACTGACCCT', b'ATTAACTGCTAAC', b'ATTGACGGCCACG', b'AATAGAGATAGCA', b'ATTCACTGCCACT', b'ATACACTCCCCCT', b'ATTCACTGCTACC', b'AATTCCAAAGTCA', b'AATTCCATAGGCA', b'ATTTACTGCTACC', b'ATGCACTGCAACC', b'ATTCACGGCAAAC', b'ATGGACTGCAACC', b'ATTCCCTGCGCCT', b'ATTTCCTGCAACC', b'AATCCCAAAGGCA', b'ATTCCCTGTGACT', b'AATAACTGCTACC', b'TTGTGCGATTGCG', b'ATTTCCAAAGGCC', b'ATTAGAGATCGCG', b'ATTGCCTGCCCCT', b'AATAGAGATACCG', b'ATTCACTGCCCCA', b'ATTGACTGCGCCA', b'ATTTGTGATAGCG', b'ATGTGAGTTAGCG', b'ATGGACTGCAAAC', b'ATGTCCTTAGGCA', 

/tmp/ipykernel_1842155/2409951786.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  deduped = todedup.apply(umicount_per_grna_and_group)
